In [1]:
%load_ext autoreload
%autoreload all

In [2]:
from contextlib import redirect_stdout
from os import devnull
import warnings

from create_model import create_model
from pymf6.mf6 import MF6
from pymf6.api import create_mutable_bc

from analytic_well_cell import AnalyticWell

In [3]:
model_path_controlled = 'models/c_100_100'
name = 'gwf_c_100_100'
mf6 = MF6(model_path_controlled + '/mfsim.nam', use_modflow_api=True)

In [4]:
gwf = mf6.models['gwf6']
wel = create_mutable_bc(gwf.wel)

/Users/mike/Dev/pymf6/.pixi/envs/analytic/lib/python3.12/site-packages/modflowapi/extensions/apimodel.py:371: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  nodeuser = np.arange(nodes).astype(int)


In [5]:
well_levels = []
with warnings.catch_warnings(action="ignore"), open(devnull, 'w') as fobj:
    with redirect_stdout(fobj):
        awell = AnalyticWell(gwf)
        for step in mf6.steps:
            well_levels.append(awell.calc_well_head(-wel.q[0]))
well_levels[-1]

(Model, 5.272882166946008)

In [6]:
from run_controlled import run_model

In [7]:
run_model()

(Model, 5.272882166946008)

In [8]:
from timeit import default_timer

In [9]:
start = default_timer()
run_model()
print(default_timer() - start)

8.771299125161022


In [10]:
start = default_timer()
run_model()
print(default_timer() - start)

8.819216167088598


In [11]:
run_times = {0.25: {'run_time': 1111.331902957987, 'well_level': 5.87527215062761},
 0.5: {'run_time': 271.3050797500182, 'well_level': 5.692237598347647},
 1: {'run_time': 68.216576582985, 'well_level': 5.984964642838974},
 5: {'run_time': 2.8074117079377174, 'well_level': 6.782194342990477},
 10: {'run_time': 0.7767317919060588, 'well_level': 7.295202624792037},
 100: {'run_time': 0.14793095807544887, 'well_level': 9.232159585697843}}

In [12]:
for size, values in reversed(run_times.items()):
    print(size, int((1000 / size) ** 2), *(round(x, 2) for x in values.values()))

100 100 0.15 9.23
10 10000 0.78 7.3
5 40000 2.81 6.78
1 1000000 68.22 5.98
0.5 4000000 271.31 5.69
0.25 16000000 1111.33 5.88


In [13]:
(1000 / 1) ** 2

1000000.0